In [4]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)

Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [5]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './eng_custom.train')
training_data.show()
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|I want to go to t...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 0, I,...|[[pos, 0, 0, 0, [...|[[named_entity, 0...|
|I want to go to t...|[[document, 0, 26...|[[document, 0, 26...|[[token, 0, 0, I,...|[[pos, 0, 0, 0, [...|[[named_entity, 0...|
|I want to go to t...|[[document, 0, 30...|[[document, 0, 30...|[[token, 0, 0, I,...|[[pos, 0, 0, 0, [...|[[named_entity, 0...|
|I want to go to t...|[[document, 0, 26...|[[document, 0, 26...|[[token, 0, 0, I,...|[[pos, 0, 0, 0, [...|[[named_entity, 0...|
|I want to go to t...|[[document, 0, 30...|[[document, 0, 30...|[[token, 0, 0, I,...|[[pos, 0, 0, 0, [..

In [6]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)



nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(100)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("test_withEmbeds.parquet")


test_data = CoNLL().readDataset(spark, './eng.testa')
test_data = bert.transform(test_data)

import os
 
# Directory name
directory = "test_withEmbeds.parquet"
 
# Parent Directory
parent = "./"
 
# Path
path = os.path.join(parent, directory)
 
# Remove the Directory
# "GeeksforGeeks"
try:
    os.rmdir(path)
    print("Directory '% s' has been removed successfully" % directory)
except OSError as error:
    print(error)
    print("Directory '% s' can not be removed" % directory)


test_data.write.parquet("test_withEmbeds.parquet")


ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]
[Errno 2] No such file or directory: './test_withEmbeds.parquet'
Directory 'test_withEmbeds.parquet' can not be removed


22/10/27 18:59:50 WARN TaskSetManager: Stage 6 contains a task of very large size (1011 KiB). The maximum recommended task size is 1000 KiB.
2022-10-27 19:01:13.626829: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 22 - training examples: 86
Epoch 1/100 started, lr: 0.003, dataset size: 86
Epoch 1/100 - 0.68s - loss: 55.915108 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 14	 0	 0.5625	 1.0	 0.72
Act	 24	 13	 0	 0.6486486	 1.0	 0.7868852
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 42	 1	 26	 0.9767442	 0.61764705	 0.7567568
Obj	 0	 1	 1	 0.0	 0.0	 0.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 84 fp: 29 fn: 29 labels: 6
Macro-average	 prec: 0.36464882, rec: 0.43627453, f1: 0.39725897
Micro-average	 prec: 0.74336284, rec: 0.74336284, f1: 0.7433629
Quality on test dataset: 
time to finish evaluation: 4.67s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 9622	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 37106	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0

Quality on test dataset: 
time to finish evaluation: 3.72s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 180	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 9625	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 39793	 0	 0.0	 0.0	 0.0
Obj	 0	 1525	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 139	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 13
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 9/100 started, lr: 0.0028846154, dataset size: 86
Epoch 9/100 - 0.08s - loss: 1.656894 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	 1.0	 1.0
Obj	 1	 0	 0	 1.0	 1.

Quality on test dataset: 
time to finish evaluation: 3.65s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 385	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 6168	 0	 0.0	 0.0	 0.0
Neg	 0	 3	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 44146	 0	 0.0	 0.0	 0.0
Obj	 0	 560	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 13
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 16/100 started, lr: 0.0027906976, dataset size: 86
Epoch 16/100 - 0.08s - loss: 1.0142235 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	 1.0	 1.0
Obj	 1	 0	 0	 1.0	 1.

...

Quality on test dataset: 
time to finish evaluation: 3.65s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 64	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 9451	 0	 0.0	 0.0	 0.0
Neg	 0	 7	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 41368	 0	 0.0	 0.0	 0.0
Obj	 0	 359	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 13	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 14
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 72/100 started, lr: 0.0022140222, dataset size: 86
Epoch 72/100 - 0.08s - loss: 0.015874323 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	 

Quality on test dataset: 
time to finish evaluation: 3.65s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 91	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 9957	 0	 0.0	 0.0	 0.0
Neg	 0	 11	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 40530	 0	 0.0	 0.0	 0.0
Obj	 0	 670	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 3	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 14
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 79/100 started, lr: 0.0021582735, dataset size: 86
Epoch 79/100 - 0.08s - loss: 0.1862329 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	 1.

Quality on test dataset: 
time to finish evaluation: 3.64s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 156	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 7005	 0	 0.0	 0.0	 0.0
Neg	 0	 11	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 43139	 0	 0.0	 0.0	 0.0
Obj	 0	 950	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 1	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 14
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 86/100 started, lr: 0.0021052633, dataset size: 86
Epoch 86/100 - 0.08s - loss: 0.014141128 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	

Quality on test dataset: 
time to finish evaluation: 3.64s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 88	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 10852	 0	 0.0	 0.0	 0.0
Neg	 0	 5	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 38819	 0	 0.0	 0.0	 0.0
Obj	 0	 1494	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 4	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 14
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 93/100 started, lr: 0.0020547945, dataset size: 86
Epoch 93/100 - 0.08s - loss: 0.007936929 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	

Quality on test dataset: 
time to finish evaluation: 3.62s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 120	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 9864	 0	 0.0	 0.0	 0.0
Neg	 0	 7	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 40760	 0	 0.0	 0.0	 0.0
Obj	 0	 502	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 9	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 14
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 100/100 started, lr: 0.002006689, dataset size: 86
Epoch 100/100 - 0.08s - loss: 0.016716408 - batches: 4
Quality on validation dataset (20.0%), validation examples = 17
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 18	 0	 0	 1.0	 1.0	 1.0
Act	 24	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 1	 0.0	 0.0	 0.0
0	 68	 0	 0	 1.0	

In [7]:
ner_model.stages[1].write().overwrite().save('NER_bert_20200221')

2022-10-27 19:08:27.070486: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [8]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)

loaded_ner_model = NerDLModel.load("NER_bert_20200221")\
   .setInputCols(["sentence", "token", "bert"])\
   .setOutputCol("ner")


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_span')

custom_ner_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence,
      tokenizer,
      bert,
      loaded_ner_model,
      ner_converter
 ])



bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


2022-10-27 19:08:33.563482: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [9]:
text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
preds = prediction_model.transform(prediction_data)
preds.show()


+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               text|            document|            sentence|               token|                bert|                 ner|            ner_span|
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|navigate to kitchen|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, na...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 12, 18, ...|
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [10]:
import pyspark.sql.functions as F
#preds.select(F.explode(F.arrays_zip('ner_span.results','ner_span.metadata')).alias("entities"))\
#    .select(F.expr("entities['0']").alias("chunk")),(F.expr("entities['1']").alias("entity"))\
#    .show(truncate=False)

In [11]:
#preds.select('token.results','ner.results').show(truncate=False)

In [12]:
from sparknlp.base import LightPipeline
text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_span': ['kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['0', '0', 'Loc'], 'bert': ['navigate', 'to', 'kitchen'], 'sentence': ['navigate to kitchen']}
['0', '0', 'Loc']
